In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pickle
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [2]:
with open('../input/train-restaurant/embedding_matrix_laptops_sentence_level.npy', 'rb') as f:
    embedding_matrix = np.load(f)

In [3]:
print(embedding_matrix.shape)

In [4]:
with open('../input/train-restaurant/train_laptops_sentence_level.json', 'rb') as f:
    data = pickle.load(f)

In [5]:
# test
with open('../input/train-restaurant/test_laptops_sentence_level.json', 'rb') as f:
    data_test = pickle.load(f)

In [6]:
sentence_level_embed = []
sentence_level_polar = []
for i in data.keys():
    sentence_level_embed.append(data[i][0])
    sentence_level_polar.append(data[i][1])
for i in data_test.keys():
    sentence_level_embed.append(data_test[i][0])
    sentence_level_polar.append(data_test[i][1])

In [7]:
print(len(sentence_level_embed))

In [8]:
x_train = np.array(sentence_level_embed[:2700])
y_train = np.array(sentence_level_polar[:2700])
x_valid = np.array(sentence_level_embed[2700:2900])
y_valid = np.array(sentence_level_polar[2700:2900])
x_test = np.array(sentence_level_embed[2900:])
y_test = np.array(sentence_level_polar[2900:])

In [9]:
print(x_train.shape)
print(y_train.shape)

In [10]:
print(x_valid.shape)
print(y_valid.shape)

In [11]:
print(x_test.shape)
print(y_test.shape)

In [12]:
! mkdir ./sentence_level_Bilstm

In [13]:
class sentence_level_Bi_lSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim_1):
        global embedding_matrix
        print(embedding_matrix.shape)
        super().__init__()
        self.hidden_dim_1 = hidden_dim_1
        self.embedding_dim = embedding_dim
        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_layer.weight.data.copy_(torch.tensor(embedding_matrix))
        self.embedding_layer.weight.requires_grad = False
        self.sentence_level_lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim_1, num_layers=3, batch_first=True, bidirectional=True)
        self.fc_layer1 = nn.Linear(2*hidden_dim_1, 3)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, input_emb_review):
        input_emb = self.embedding_layer(input_emb_review)
        sent_lstm_out, hid = self.sentence_level_lstm(input_emb)
        fc_out = self.fc_layer1(sent_lstm_out)
        return self.softmax(fc_out)

In [14]:
# torch.autograd.set_detect_anomaly(True)
sentence_level_Bilstm = sentence_level_Bi_lSTM(embedding_matrix.shape[0], embedding_matrix.shape[1],150).double().cuda()
optimizer = optim.SGD(sentence_level_Bilstm.parameters(), lr=0.1)
loss_function = nn.CrossEntropyLoss()
num_epochs = 100
prev_accuracy = 0
accuracy_train = []
accuracy_valid = []
loss_value = []
prev_valid_acc = 0
accuracy_score_valid = 0
for epoch in range(num_epochs):
    print("\n\nEpoch:", epoch+1)
    current_loss = 0
    sentence_level_Bilstm.train()
    count = 0
    accuracy = 0
    accuracy_valid_score = 0
    accuracy_train_score = 0
    count_valid = 0
    for i in tqdm(range(0, len(x_train), 5)):
        review = torch.tensor(x_train[i:i+5]).cuda()
        output = sentence_level_Bilstm(review).cuda()
        output = output[:,output.shape[1]-1,:]
        y_label = torch.tensor(y_train[i:i+5], dtype=torch.long).cuda()
        loss = loss_function(output, y_label)
        y_label = y_label.cpu()
        current_loss += loss.item()
        final_result = np.array(torch.argmax(output, dim=1).cpu().clone())
        accuracy += accuracy_score(final_result, y_label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1
    for i in tqdm(range(0, len(x_valid), 5)):
        review = torch.tensor(x_valid[i:i+5]).cuda()
        output = sentence_level_Bilstm(review).cuda()
        output = output[:,output.shape[1]-1,:]
        y_label = torch.tensor(y_valid[i:i+5], dtype=torch.long).cuda()
        y_label = y_label.cpu()
        final_result = np.array(torch.argmax(output, dim=1).cpu().clone())
        accuracy_valid_score += accuracy_score(final_result, y_label)
        count_valid += 1
#         break
    accuracy_score_train = accuracy / count
    valid_score = accuracy_valid_score / count_valid 
    print("Loss is", current_loss)
    print("Accuracy is", accuracy_score_train, "%")
    print("Validation accuracy is", valid_score, "%")
    accuracy_train.append(accuracy_score_train)
    loss_value.append(current_loss/count)
    accuracy_valid.append(valid_score)
    if prev_valid_acc < accuracy_score_valid:
        torch.save(sentence_level_Bilstm, './sentence_level_Bilstm/new-model-' + str(epoch+1) + '.pt')
    prev_accuracy = accuracy
    prev_valid_acc = accuracy_score_valid
#     break
# print(current_loss)

In [15]:
torch.save(sentence_level_Bilstm, './sentence_level_Bilstm/sentence_level_new-model-' + str(epoch+1) + '.pt')

In [16]:
import matplotlib.pyplot as plt
plt.plot(accuracy_train)
plt.title("Training Accuracy VS Epoch for sentence-level")
plt.xlabel("Epoch")
plt.ylabel("Accuracy fraction")
plt.savefig('./sentence_level_Bilstm/train_acc.jpg')
plt.show()

In [17]:
plt.plot(loss_value)
plt.title("Loss VS Epoch for sentence-level")
plt.ylabel("Average Loss")
plt.xlabel("Epoch")
plt.savefig('./sentence_level_Bilstm/loss.jpg')
plt.show()

In [18]:
plt.plot(accuracy_valid)
plt.title("Validation Accuracy VS Epoch for sentence-level")
plt.xlabel("Epoch")
plt.ylabel("Accuracy fraction")
plt.savefig('./sentence_level_Bilstm/valid_acc.jpg')
plt.show()

In [19]:
import shutil
shutil.make_archive("sentence_level_Bilstm", 'zip', "./sentence_level_Bilstm")

In [20]:
import os
os.chdir(r'/kaggle/working/')
from IPython.display import FileLink
FileLink(r'./sentence_level_Bilstm.zip')

In [21]:
# model = torch.load("/kaggle/working/glove_300/new-model-15.pt")
accuracy_test_score = 0
count_test = 0
for i in tqdm(range(0, len(x_test), 10)):
        review = torch.tensor(x_test[i:i+10]).cuda()
        output = sentence_level_Bilstm(review).cuda()
        output = output[:,output.shape[1]-1,:]
        y_label = torch.tensor(y_test[i:i+10], dtype=torch.long).cuda()
        y_label = y_label.cpu()
        final_result = np.array(torch.argmax(output, dim=1).cpu().clone())
        accuracy_test_score += accuracy_score(final_result, y_label)
        count_test += 1
test_score = accuracy_test_score / count_test
print("Test Accuracy:", test_score)